In [1]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
import sys
from selenium.webdriver.chrome.options import Options
import json

In [ ]:
# 設定瀏覽器不會跳出通知alert
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
# Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 1 
})

In [2]:
def LoginFacebook():
    # Step 1) Navigate to Facebook
    driver.get("http://www.facebook.com")
    # Step 2) Search & Enter the Email or Phone field & Enter Password
    username = driver.find_element_by_id("email")
    password = driver.find_element_by_id("pass")
    try:
        submit   = driver.find_element_by_id("loginbutton")
    except:
        submit   = driver.find_element_by_id("login")
    username.send_keys("your email")
    password.send_keys("your password")
    # Step 3) Click Login
    submit.click()

In [3]:
def FindLinks(url, n):
    Links = []
    driver.get(url)
    
    for i in range(n):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    
    for i in posts:
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
        
    return Links

In [4]:
def expand(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    try:
        driver.find_element_by_xpath('//a[@lang="en"]').click()
    except:
        print("Now is in EN_US")
    try:
        driver.find_element_by_xpath('//i[@class="img sp_77DmwRIU05-_2x sx_3abbb7"]').click()
        print('close photo page')
    except:
        print('not photo page')
    try: 
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]')
        print('Comments')
    except:
        print('No comments',sys.exc_info())
        print('-----')
    k = 1
    while k != 0:
        k = 0
        element = driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//div[@data-testid="UFI2CommentsList/root_depth_0"]')
        buttons =  element.find_elements_by_xpath('.//a[@role="button"]')
        buttons.reverse()
        for i in buttons[:3]: 
            if(i.text =='View more comments'):
                time.sleep(1)
                try:
                    i.click()
                    print('Opening "%s"' % i.text)
                except:
                    print('No more "View more comments"')
                time.sleep(1)
                k += 1 
                
#展開See More
    for s in buttons:
        if(s.text == 'See More'):
            time.sleep(1)
            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", s)
                time.sleep(1)
                print('Opening "%s"' % s.text)
                s.click()
            except:
                print('e:"',sys.exc_info())
    print('No more "See More"')

In [5]:
# # 讚
# _2p78 _2p7a _9-- img sp_4AaFE3LL7KI_2x sx_86ed03
# # 哈
# _2p78 _2p7a _9-- img sp_4AaFE3LL7KI_2x sx_fd6e48
# # 哇
# _2p78 _2p7a _9-- img sp_4AaFE3LL7KI_2x sx_b94e28
# # 嗚
# _2p78 _2p7a _9-- img sp_4AaFE3LL7KI_2x sx_9c8d9d
# # 心
# _2p78 _2p7a _9-- img sp_4AaFE3LL7KI_2x sx_e11927
# # 怒
# _2p78 _2p7a _9-- img sp_4AaFE3LL7KI_2x sx_526521

def Reaction_Swither(r):
    if(r=='sx_86ed03'):
        Reaction = 'Like'
    if(r=='sx_fd6e48'):
        Reaction = 'Haha'
    if(r=='sx_b94e28'):
        Reaction = 'Wow'
    if(r=='sx_9c8d9d'):
        Reaction = 'Sad'
    if(r=='sx_e11927'):
        Reaction = 'Love'
    if(r=='sx_526521'):
        Reaction = 'Angry'
    return Reaction

In [6]:
# 文章內容與互動摘要
def PostContent(soup):
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'clearfix d_rh-0u2036'})
    # 互動摘要區(讚、留言與分享)
#     feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = PosterInfo.find('a', {'class':'_5pb8 m_rh-0tzwj_ _8o _8s lfloat _ohe'}).attrs['href'].split('/?',2)[0].split('/',-1)[-1]
    print(ID)
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'})
    # 文章內容
    try:
        Content = userContent.find('div', {'data-testid':'post_message'}).text
    except:
        Content = 'None'
    print('Name:{}\nID:{}\nLink:{}\nTime:{}\nContent:{}'.format(Name, ID, Link, Time, Content))
    # Like
#     try:
#         Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label']
#     except:
#         Like = '0' 
#     # Angry
#     try:
#         ANGER = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label']
#     except:
#         ANGER = '0'
#     # HAHA
#     try:
#         HAHA = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label']
#     except:
#         HAHA = '0'
#     # LOVE
#     try:
#         LOVE = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label']
#     except:
#         LOVE = '0'
#     # SORRY
#     try:
#         SORRY = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label']
#     except:
#         SORRY = '0'
#     # WOW
#     try:
#         WOW = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label']
#     except:
#         WOW = '0'
#     # 留言
#     try:
#         commentcount = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text
#     except:
#         commentcount = '0' 
#     # 分享
#     try:
#         share = feedback.find('span', {'class':'_355t _4vn2'}).text
#     except:
#         share = '0' 
#     print(Name,ID,Link,Time,Content,Like,ANGER,HAHA,LOVE,SORRY,WOW,commentcount,share)
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content
                 }],
        columns = ['Name', 'ID', 'Link', 'Time', 'Content'])

In [7]:
# 留言
def CrawlComment(soup):
    Comments = pd.DataFrame()
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 回應貼文的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_0'}):
        try:
            CommentContent = i.find('span', {'dir':'ltr'}).text
        except:
            CommentContent = 'Sticker'
            
        Comment = pd.DataFrame(data = [{'CommentID':i.find('a', {'class':'_3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'CommentName':i.find('img').attrs['alt'],
                                 'CommentTime':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'CommentContent':CommentContent,
                                 'Link':driver.current_url}],
                        columns = ['CommentID', 'CommentName', 'CommentTime', 'CommentContent', 'Link'])

        Comments = pd.concat([Comments, Comment], ignore_index=True)
   
    return Comments



In [8]:
# 心情
def CrawlReactions(soup):
    Reactions = pd.DataFrame()
    # 按下跳出留言者列表的視窗
    windows =  driver.find_element_by_xpath('//a[@class="_3dlf"]')
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", windows)
    time.sleep(1)
    windows.click()
    time.sleep(1)
    k = 1
    while k != 0:
        k = 0
        time.sleep(3)
        try:
            more = driver.find_element_by_xpath('//div[@class="clearfix mtm uiMorePager stat_elem _52jv"]//a[@class="pam uiBoxLightblue uiMorePagerPrimary"]')
        except:
            more = None
        if more is not None:
            try:
                more.click()
                print('Opening')
            except:
                print('No more')
            k += 1 
    
    # 爬取 ID & reaction
    comment_list = driver.find_elements_by_xpath('//li[@class="_5i_q"]')
    print(len(comment_list),"筆留言")
    for i in comment_list:
        ID = i.find_element_by_xpath('.//div[@class="_5j0e fsl fwb fcb"]//a').get_attribute('data-gt').split(':')[4].split(',')[0].split('\"')[1]
        r = i.find_element_by_xpath('.//div[@class="_2ar3"]//i[@data-testid="ufiReactionsIconsTestId"]').get_attribute('class').split()[-1]
        Reaction = Reaction_Swither(r)
        
        CommentReaction = pd.DataFrame(data = [{'ID':ID,
                                 'Reaction':Reaction}],
                        columns = ['ID', 'Reaction'])
        Reactions = pd.concat([Reactions, CommentReaction], ignore_index=True)
    return Reactions

In [ ]:
# ----------------------------Func----------------------------

In [1095]:
# 開啟瀏覽器
driver = webdriver.Chrome(chrome_options=option, executable_path='./cdriver')
# 登入
LoginFacebook()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [33]:
# Main

# 計時花費時間
start_time = time.time()



# 取得貼文連結
Links=[]
Links.append('https://www.facebook.com/tsaiingwen/')  



# 建立DF
PostsInformation = pd.DataFrame()
PostsComments = pd.DataFrame()
PostsReactions = pd.DataFrame()

for i in Links:
    print('Dealing with: ' + i)
    try:
        expand(i)
        soup = BeautifulSoup(driver.page_source)
        PostsInformation = pd.concat([PostsInformation, PostContent(soup)],ignore_index=True)
        PostsComments = pd.concat([PostsComments, CrawlComment(soup)],ignore_index=True)
        PostsReactions = pd.concat([PostsReactions, CrawlReactions(soup)],ignore_index=True)
    except:
        print('Load Failed: ' ,sys.exc_info())

PostsInformation
PostsComments
PostsReactions

print("--- %s seconds ---" % (time.time() - start_time))

Dealing with: https://www.facebook.com/ChiangDaSport/photos/a.1062321090490228/2490645604324429/?type=1&theater
Now is in EN_US
close photo page
Comments
Opening "View more comments"
Opening "View more comments"
Opening "View more comments"
Opening "View more comments"
No more "View more comments"
No more "See More"
ChiangDaSport
Name:強大體育用品
ID:ChiangDaSport
Link:https://www.facebook.com/ChiangDaSport/photos/a.1062321090490228/2490645604324429/?type=1&theater
Time:11/6/19, 8:28 PM
Content:張奕，今天最帥的男人！ 除了贊助大將軍抽獎，加碼再抽一波一打空白簽名球 今天贏球，強大抽球 大家說好不好啊 ＃台灣加油 ＃刷一波台灣加油
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
Opening
891 筆留言
--- 268.1948699951172 seconds ---


In [34]:
PostsInformation.to_excel('PostsInformation.xlsx')
PostsComments.to_excel('PostsComments.xlsx')
PostsReactions.to_excel('PostsReactions.xlsx')